In [1]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os
import pandas as pd
import numpy as np
from filters import filter

load_dotenv()
engine = create_engine(f"postgresql://{os.environ['DB_USERNAME']}:{os.environ['DB_PASSWORD']}@{os.environ['DB_HOST']}:{os.environ['DB_PORT']}/{os.environ['DB_DATABASE']}")

In [2]:
#df_filter = pd.read_sql_table(table_name="game", con=engine)
df_filter = pd.read_csv('./game_filter.csv')

In [3]:
df_filter

,id,name,platform,rating,num_of_player_idx,img_url,major_genre
0,1,#DRIVE,Switch,all,1,https://static.metacritic.com/images/products/...,"Racing, Arcade"
1,2,#IDARB,Xbox One,all,5,https://static.metacritic.com/images/products/...,"Arcade, Action"
2,3,#KILLALLZOMBIES,Xbox One,13+,1,https://static.metacritic.com/images/products/...,"Shooting, Action"
3,4,'Splosion Man,Xbox 360,10+,4,https://static.metacritic.com/images/products/...,Action
4,5,.detuned,PlayStation 3,all,1,https://static.metacritic.com/images/products/...,"Arcade, Action"
...,...,...,...,...,...,...,...
12899,12900,rain,PlayStation 3,10+,1,https://static.metacritic.com/images/products/...,"Action, Arcade, Adventure"
12900,12901,theHunter: Call of the Wild,PC,13+,5,https://static.metacritic.com/images/products/...,"Sports, Shooting"
12901,12902,uDraw Studio,Wii,all,5,https://static.metacritic.com/images/products/...,NaN
12902,12903,void tRrLM(); //Void Terrarium,Switch,13+,1,https://static.metacritic.com/images/products/...,Arcade


In [4]:
# input example
age = 20             # 9 10 13 17 20
platform = ["PC", "PlayStation 4"]
players = 1
major_genre = ["Tactics", "Puzzle"]
tag = [1, 0, 0, 0, 0, 1, 1]   # graphcis  sound  creativity  freedom  hitting  completion  difficulty 
games = ["Zombie Driver: Immortal Edition", "Zumba Fitness Rush"]

In [5]:
df_filter.dtypes

id                    int64
name                 object
platform             object
rating               object
num_of_player_idx     int64
img_url              object
major_genre          object
dtype: object

In [6]:
# 필터링
idx = filter(df_filter, age, platform, players, major_genre)

In [7]:
idx

20          21
32          33
42          43
58          58
85          85
         ...  
12853    12854
12854    12855
12862    12863
12884    12885
12886    12887
Name: id, Length: 675, dtype: int64

In [8]:
df_cb = pd.read_sql_table(table_name="cb_model", con=engine)

In [9]:
df_cb

,id,genre,graphics,sound,creativity,freedom,hitting,completion,difficulty
0,1,"Racing, Arcade, Automobile",1,1,1,1,1,1,0
1,2,"Action, Miscellaneous, General, Party / Minigame",1,1,1,1,1,1,0
2,3,"Action, Shooter, Shoot-Em-Up, Top-Down",1,1,0,0,0,0,0
3,4,"Action, General, Platformer, Platformer, 2D, 2D",1,1,1,1,1,1,1
4,5,"Action, Miscellaneous, Rhythm, Music, Music",0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
12899,12900,"Miscellaneous, Action Adventure, Fantasy, Gene...",1,1,1,0,0,1,0
12900,12901,"Sports, Individual, Nature, Hunting",1,1,0,0,0,0,1
12901,12902,"Miscellaneous, General, General",0,0,0,0,0,0,0
12902,12903,"Role-Playing, Roguelike",0,0,0,0,0,0,0


In [10]:
df_user = pd.DataFrame(columns=['id', 'genre', 'graphics', 'sound', 'creativity', 'freedom', 'hitting', 'completion', 'difficulty'])

for i in games:
    input_idx = df_filter[df_filter['name'] == i].index
    input_df =  df_cb.loc[input_idx]
    df_user = pd.concat([df_user, input_df[['id', 'genre']]], ignore_index=True)

df_user = df_user.fillna(dict(zip(df_user.columns[2:], tag)))

In [11]:
df_user

,id,genre,graphics,sound,creativity,freedom,hitting,completion,difficulty
0,12833,"Action, Simulation, General, Vehicle, Combat",1,0,0,0,0,1,1
1,12864,"Miscellaneous, Rhythm, Exercise / Fitness, Dan...",1,0,0,0,0,1,1


In [12]:
df_cb = df_cb[df_cb['id'].isin(idx)]

In [13]:
df_cb

,id,genre,graphics,sound,creativity,freedom,hitting,completion,difficulty
20,21,"Miscellaneous, Puzzle, Action",0,0,0,0,0,0,0
32,33,"Strategy, General",1,1,0,0,0,1,0
42,43,"General, Puzzle, Miscellaneous, Adventure, Puz...",1,1,0,0,1,0,0
57,58,"Action, Beat-Em-Up, Beat-Em-Up, 2D",0,0,0,0,0,0,0
84,85,"Miscellaneous, Compilation",0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
12853,12854,"Strategy, Tycoon, Management, Tycoon, Business...",1,0,1,0,0,0,0
12854,12855,"Strategy, Tycoon, Management, Tycoon, Business...",1,0,1,1,1,1,0
12862,12863,"Miscellaneous, Puzzle, Action, Puzzle, Action",1,1,1,0,0,1,0
12884,12885,"Puzzle, Action",0,0,0,0,0,0,0


In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.concat([df_cb, df_user], ignore_index=True)

# "genre" 열의 장르들을 숫자로 매핑
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '))
genre_matrix = vectorizer.fit_transform(df['genre'])

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [16]:
genre_df = pd.DataFrame(genre_matrix.toarray())

In [17]:
genre_df

,0,1,2,3,4,5,6,7,8,9,...,103,104,105,106,107,108,109,110,111,112
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
673,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
674,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
675,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [18]:
# "id" 열 제외
df_numeric = df.drop(['id', 'genre'], axis=1)

# 숫자 데이터와 장르 데이터 결합
df_combined = pd.concat([df_numeric, genre_df], axis=1)

In [19]:
df_combined

,graphics,sound,creativity,freedom,hitting,completion,difficulty,0,1,2,...,103,104,105,106,107,108,109,110,111,112
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
675,1,0,0,0,0,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [20]:
# 코사인 유사도 계산
similarity_matrix = cosine_similarity(df_combined)

# 유사도 행렬을 데이터프레임으로 변환
similarity_df = pd.DataFrame(similarity_matrix, index=df.index, columns=df.index)

In [21]:
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,667,668,669,670,671,672,673,674,675,676
0,1.000000,0.000000,0.462910,0.235702,0.408248,0.000000,0.000000,0.160128,0.333333,0.218218,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.800641,0.816497,0.816497,0.204124,0.218218
1,0.000000,1.000000,0.478091,0.000000,0.000000,0.316228,0.632456,0.248069,0.000000,0.338062,...,0.404520,0.169031,0.516398,0.298142,0.387298,0.372104,0.000000,0.000000,0.474342,0.338062
2,0.462910,0.478091,1.000000,0.000000,0.188982,0.566947,0.472456,0.148250,0.000000,0.202031,...,0.161165,0.000000,0.154303,0.089087,0.154303,0.518875,0.377964,0.377964,0.283473,0.202031
3,0.235702,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.113228,0.235702,0.154303,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.226455,0.288675,0.288675,0.144338,0.000000
4,0.408248,0.000000,0.188982,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.196116,0.000000,0.000000,0.000000,0.267261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,0.800641,0.372104,0.518875,0.226455,0.196116,0.000000,0.196116,0.384615,0.320256,0.419314,...,0.250873,0.000000,0.320256,0.184900,0.240192,1.000000,0.784465,0.784465,0.392232,0.314485
673,0.816497,0.000000,0.377964,0.288675,0.000000,0.000000,0.000000,0.196116,0.408248,0.267261,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.784465,1.000000,1.000000,0.250000,0.000000
674,0.816497,0.000000,0.377964,0.288675,0.000000,0.000000,0.000000,0.196116,0.408248,0.267261,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.784465,1.000000,1.000000,0.250000,0.000000
675,0.204124,0.474342,0.283473,0.144338,0.000000,0.250000,0.375000,0.294174,0.204124,0.267261,...,0.213201,0.000000,0.204124,0.117851,0.204124,0.392232,0.250000,0.250000,1.000000,0.400892


In [31]:
# 상위 5개 유사한 항목 찾기
item_id = 0  # 기준 항목의 인덱스
similar_items = similarity_df[item_id].nlargest(len(df_user) + 5)[len(df_user):]  # 상위 5개 유사한 항목 (자기 자신 제외)

In [32]:
# 상위 5개 유사한 항목의 인덱스
similar_item_ids = similar_items.index.tolist()


# 추천 게임 목록 생성
recommendations = df.loc[similar_item_ids, 'id'].tolist()


In [33]:
recommendations

[12887, 12863, 8266, 9892, 8737]

In [34]:
# 결과 출력
print(f"\n추천하는 게임:")
for i, game in enumerate(recommendations):
    print(f"{i+1}. {game}")


추천하는 게임:
1. 12887
2. 12863
3. 8266
4. 9892
5. 8737


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 기존 게임 데이터프레임과 유저 게임 데이터프레임 통합
combined_df = pd.concat([df_cb, df_user], ignore_index=True)

# TF-IDF 벡터화 객체 생성
vectorizer = TfidfVectorizer()

# 게임 장르 데이터를 TF-IDF 벡터로 변환
genre_vectors = vectorizer.fit_transform(combined_df['genre'])

In [ ]:
combined_df

,id,genre,graphics,sound,creativity,freedom,hitting,completion,difficulty
0,21,"Miscellaneous, Puzzle, Action",0,0,0,0,0,0,0
1,33,"Strategy, General",1,1,0,0,0,1,0
2,43,"General, Puzzle, Miscellaneous, Adventure, Puz...",1,1,0,0,1,0,0
3,58,"Action, Beat-Em-Up, Beat-Em-Up, 2D",0,0,0,0,0,0,0
4,85,"Miscellaneous, Compilation",0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
672,12863,"Miscellaneous, Puzzle, Action, Puzzle, Action",1,1,1,0,0,1,0
673,12885,"Puzzle, Action",0,0,0,0,0,0,0
674,12887,"Puzzle, Action",0,0,0,0,0,0,0
675,12833,"Action, Simulation, General, Vehicle, Combat",1,0,0,0,0,1,1


In [ ]:
# 각 컬럼들 간의 코사인 유사도 계산
similarity_matrix = cosine_similarity(combined_df.drop(['id', 'genre'], axis=1), genre_vectors)

# 코사인 유사도 결과를 데이터프레임으로 변환
similarity_df = pd.DataFrame(similarity_matrix, columns=combined_df['id'], index=combined_df.columns[2:])

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 7 while Y.shape[1] == 138

In [ ]:
similarities_genre = cosine_similarity(genre_vectors[-len(df_user):], genre_vectors[:-len(df_user)])
similarities_tag = cosine_similarity(genre_vectors[-len(df_user):], genre_vectors[:-len(df_user)])

In [ ]:
# 추천을 위한 유사도 측정
similarities = cosine_similarity(genre_vectors[-2:], genre_vectors[:-2])
top_similar_indices = similarities.argsort()[0][::-1][:5] #5개 추천

# 추천 게임 목록 생성
recommendations = combined_df.loc[top_similar_indices, 'id'].tolist()

# 결과 출력
print(f"\n추천하는 게임:")
for i, game in enumerate(recommendations):
    print(f"{i+1}. {game}")


추천하는 게임:
1. 3447
2. 8263
3. 5324
4. 3265
5. 3266


In [ ]:
"""def age_filter(df, age):
    if age == 9:
        return df[df['rating'] == 'all']
    elif age == 10:
        return df[(df['rating'] == 'all') | (df['rating'] == '10+')]
    elif age == 13:
        return df[(df['rating'] == 'all') | (df['rating'] == '10+') | (df['rating'] == '13+')]
    elif age == 17:
        return df[(df['rating'] == 'all') | (df['rating'] == '10+') | (df['rating'] == '13+') | (df['rating'] == '17+')]
    else: # age == 20:
        return df
    
    
def players_filter(df, players):
    return df[df['num_of_player_idx'] == players]


def platform_and_genre_filter(df_, platform, major_genre):
    df = df_.copy()
    df['split_genres'] = df['major_genre'].apply(lambda genres: genres.split(','))
    df['split_platform'] = df['platform'].apply(lambda genres: genres.split(','))

    idx_arr = []
    for genre, platforms, idx in zip(df['split_genres'],df['split_platform'], df['id']):
        i_ = [x.strip() for x in genre]
        j_ = [y.strip() for y in platform]

        if any(x in i_ for x in major_genre) and any(y in j_ for y in platform): idx_arr.append(idx-1)

    # 'split_genres' 열을 삭제
    df = df.drop('split_genres', axis=1)
    df = df.drop('split_platform', axis=1)
    return df.loc[idx_arr]
    

def major_genre_filter(df_, major_genre):
    # major_genre 열에서 각 장르를 개별적으로 추출하여 새로운 열인 'split_genres'에 저장
    df = df_.copy()
    df['split_genres'] = df['major_genre'].apply(lambda genres: genres.split(','))
    
    idx_arr = []
    for genre, idx in zip(df['split_genres'], df['id']):
        i_ = [x.strip() for x in genre]
        if any(x in i_ for x in major_genre): idx_arr.append(idx-1)

    # 'split_genres' 열을 삭제
    #df = df.drop('split_genres', axis=1)
    
    return df_.loc[idx_arr]
    

def filter (df, age, platform, players, major_genre):
    output = platform_and_genre_filter(df, platform, major_genre)
    output = players_filter(output, players)
    output = age_filter(output, age)

    return output"""